In [1]:
import os
from os.path import isfile
import sys
import glob

os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler

import mlflow
import mlflow.tensorflow

2024-09-29 19:30:05.410466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 19:30:05.523471: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("ATMS+CPL q")

2024/09/29 19:30:09 INFO mlflow.tracking.fluent: Experiment with name 'ATMS+CPL q' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///data/nature_run/work/src/mlruns/892757010930948959', creation_time=1727652609678, experiment_id='892757010930948959', last_update_time=1727652609678, lifecycle_stage='active', name='ATMS+CPL q', tags={}>

In [3]:
def get_data(filelist):
    hsel_val = []
    scalar_val = []
    table_val = []
    cpl_val = []

    for i in filelist:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        hsel_val.append(tmp["mh"])
        scalar_val.append(tmp["scalar"])
        table_val.append(tmp["table"])
        cpl_val.append(tmp["cpl"])

    return np.vstack(hsel_val), np.vstack(cpl_val), np.vstack(scalar_val), np.vstack(table_val)


train = ['/data/nature_run/fulldays_reduced/all_cpl_20060815.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060915.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20061015.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060515.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060715.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20061115.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060315.npz', 
         '/data/nature_run/fulldays_reduced/all_cpl_20061215.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060615.npz']

test = ['/data/nature_run/fulldays_reduced/all_cpl_20060803.npz']

validation = ['/data/nature_run/fulldays_reduced/all_cpl_20060803.npz']

hsel_train, cpl_train, scalar_train, table_train = get_data(train)
hsel_test, cpl_test, scalar_test, table_test = get_data(test)
hsel_val, cpl_val, scalar_val, table_val = get_data(validation)



In [ ]:
#hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")

In [ ]:
#hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")
#hsel_val = hsel_test.copy()
#table_val = table_test.copy()
#scalar_val = scalar_test.copy()


In [4]:
print(hsel_train.shape)
print(cpl_train.shape)
print(table_train.shape)
print(scalar_train.shape)

(8414706, 22)
(8414706, 733)
(8414706, 72, 3)
(8414706, 44)


In [5]:
# What should we do
# 1) Only use train scalars
# 2) Scale seperateley


# Presssure ####
spress_test = scalar_test[:, 3].reshape(-1, 1)
spress_train = scalar_train[:, 3].reshape(-1, 1)
spress_val = scalar_val[:, 3].reshape(-1, 1)

mins = np.min(spress_train, axis=0)
maxs = np.max(spress_train, axis=0)
np.savez("spress_scalar_cpl.npz", mins=mins, maxs=maxs)

spress_test = (spress_test - mins)/(maxs - mins)
spress_train = (spress_train - mins)/(maxs - mins)
spress_val = (spress_val - mins)/(maxs - mins)


# Labels ####
q_train = table_train[:, :, 2]
q_test = table_test[:, :, 2]
q_val = table_val[:, :, 2]

# Spectra ####
mins = np.min(hsel_train, axis=0)
maxs = np.max(hsel_train, axis=0)

np.savez("minimac_scaling_factors_cpl_mh.npz", maxs=maxs, mins=mins)

hsel_train = np.nan_to_num(hsel_train)
hsel_test = np.nan_to_num(hsel_test)
hsel_val = np.nan_to_num(hsel_val)

hsel_train = (hsel_train - mins)/(maxs - mins)
hsel_test = (hsel_test - mins)/(maxs - mins)
hsel_val = (hsel_val - mins)/(maxs - mins)

# CPL #####
mins = np.min(cpl_train, axis=0)
maxs = np.max(cpl_train, axis=0)
np.savez("minimac_scaling_factors_cpl_cpl.npz", maxs=maxs, mins=mins)

cpl_train = (cpl_train - mins)/(maxs - mins)
cpl_test = (cpl_test - mins)/(maxs - mins)
cpl_val = (cpl_val - mins)/(maxs - mins)

with tf.device('/cpu:0'):
    # train
    x_train = {'rad': tf.convert_to_tensor(hsel_train, np.float32),
               'spress': tf.convert_to_tensor(spress_train, np.float32),
               'cpl': tf.convert_to_tensor(cpl_train, np.float32)}
    del hsel_train
    del spress_train
    
    y_train =  tf.convert_to_tensor(q_train, np.float32)
    del q_train

    
    # Val
    x_val = {'rad': tf.convert_to_tensor(hsel_val, np.float32),
             'spress': tf.convert_to_tensor(spress_val, np.float32),
             'cpl': tf.convert_to_tensor(cpl_val, np.float32)}
    del hsel_val
    del spress_val
    
    y_val =  tf.convert_to_tensor(q_val, np.float32)
    del q_val

    
    # Test
    x_test = {'rad': tf.convert_to_tensor(hsel_test, np.float32),
              'spress': tf.convert_to_tensor(spress_test, np.float32),
              'cpl': tf.convert_to_tensor(cpl_test, np.float32)}
    del hsel_test
    del spress_test
    
    y_test =  tf.convert_to_tensor(q_test, np.float32)
    del q_test


2024-09-29 19:35:04.513417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 19:35:05.058197: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2024-09-29 19:35:05.059203: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31017 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1b:00.0, compute capability: 7.0


In [ ]:
# print(x_test)
# print(x_val)
# print(y_train.shape)
#a = np.argwhere(np.isnan(x_train['rad']))
#print(np.sum(np.isnan(x_train['rad'])))
#np.nan_to_num(x_train['rad'], copy=False)
#print(np.sum(np.isnan(x_train['rad'])))
#print(a)
#print(len(a))

np.sum(np.isnan(x_train['rad']))

In [6]:
def build_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]
    
    x = Concatenate()([mh, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="q")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 22,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_model(config)
model.summary()


Model: "q"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 22)]         0           []                               
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 23)           0           ['rad[0][0]',                    
                                                                  'spress[0][0]']                 
                                                                                                  
 dense (Dense)                  (None, 64)           1536        ['concatenate[0][0]']            

In [7]:
def build_cpl_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    cpl = Input(shape=(733,), name="cpl")
    # [ha, hb, hc, hd, hw, mh]

    cpl_reduced = Dense(128, activation=config["activation"])(cpl)
    cpl_reduced = Dense(32, activation=config["activation"])(cpl_reduced)

    
    x = Concatenate()([mh, spress, cpl_reduced])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress, cpl], outputs=outputs, name="q")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 22,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_cpl_model(config)
model.summary()


Model: "q"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cpl (InputLayer)               [(None, 733)]        0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 128)          93952       ['cpl[0][0]']                    
                                                                                                  
 rad (InputLayer)               [(None, 22)]         0           []                               
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  

In [8]:
print("DING")

DING


In [ ]:
# CPL

for _ in range(100):
    config = {'shape': 22,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}
    
    model = build_cpl_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=8, verbose=0)
        mlflow.log_param("CPL", True)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            verbose=2,
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100, verbose=0)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100, verbose=0)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")



2024/09/29 21:22:37 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 21:22:37 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 46s - loss: 4.7007e-04 - val_loss: 3.4588e-04 - 46s/epoch - 11ms/step
Epoch 2/1000
4208/4208 - 48s - loss: 3.4093e-04 - val_loss: 3.2523e-04 - 48s/epoch - 12ms/step
Epoch 3/1000
4208/4208 - 39s - loss: 3.2713e-04 - val_loss: 3.2417e-04 - 39s/epoch - 9ms/step
Epoch 4/1000
4208/4208 - 38s - loss: 3.1834e-04 - val_loss: 3.1432e-04 - 38s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 39s - loss: 3.1277e-04 - val_loss: 3.0830e-04 - 39s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 38s - loss: 3.0619e-04 - val_loss: 2.9790e-04 - 38s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 38s - loss: 3.0130e-04 - val_loss: 2.9066e-04 - 38s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 37s - loss: 2.9638e-04 - val_loss: 2.9421e-04 - 37s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 37s - loss: 2.9257e-04 - val_loss: 2.8206e-04 - 37s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 37s - loss: 2.8907e-04 - val_loss: 2.8289e-04 - 37s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 35s - loss: 2.8584e-04 - val_lo

2024/09/29 21:50:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/data/jmackin1/miniconda/envs/tf-gpu/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/09/29 21:54:04 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 21:54:04 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 41s - loss: 4.2716e-04 - val_loss: 3.5757e-04 - 41s/epoch - 10ms/step
Epoch 2/1000
4208/4208 - 39s - loss: 3.3399e-04 - val_loss: 3.3804e-04 - 39s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 39s - loss: 3.1276e-04 - val_loss: 3.0785e-04 - 39s/epoch - 9ms/step
Epoch 4/1000
4208/4208 - 39s - loss: 2.9772e-04 - val_loss: 2.9164e-04 - 39s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 39s - loss: 2.8398e-04 - val_loss: 2.7801e-04 - 39s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 39s - loss: 2.7635e-04 - val_loss: 2.7054e-04 - 39s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 40s - loss: 2.7108e-04 - val_loss: 2.6538e-04 - 40s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 39s - loss: 2.6760e-04 - val_loss: 2.7754e-04 - 39s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 39s - loss: 2.6529e-04 - val_loss: 2.6694e-04 - 39s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 38s - loss: 2.6238e-04 - val_loss: 2.6726e-04 - 38s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 40s - loss: 2.6017e-04 - val_los

2024/09/29 22:25:21 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 22:25:21 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


...

Epoch 1/1000
4208/4208 - 41s - loss: 9.3017e-04 - val_loss: 9.0240e-04 - 41s/epoch - 10ms/step
Epoch 2/1000
4208/4208 - 39s - loss: 8.7447e-04 - val_loss: 9.1602e-04 - 39s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 40s - loss: 8.7434e-04 - val_loss: 9.0726e-04 - 40s/epoch - 10ms/step
Epoch 4/1000
4208/4208 - 39s - loss: 8.7458e-04 - val_loss: 8.9998e-04 - 39s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 39s - loss: 8.7504e-04 - val_loss: 9.1463e-04 - 39s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 40s - loss: 8.7497e-04 - val_loss: 9.0144e-04 - 40s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 39s - loss: 8.7509e-04 - val_loss: 9.1472e-04 - 39s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 39s - loss: 8.7505e-04 - val_loss: 9.1477e-04 - 39s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 39s - loss: 8.7504e-04 - val_loss: 8.9401e-04 - 39s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 39s - loss: 8.7508e-04 - val_loss: 8.9907e-04 - 39s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 38s - loss: 8.7512e-04 - val_lo

2024/09/30 06:10:52 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/30 06:10:52 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 35s - loss: 7.6772e-04 - val_loss: 3.8220e-04 - 35s/epoch - 8ms/step
Epoch 2/1000
4208/4208 - 35s - loss: 4.0415e-04 - val_loss: 3.6867e-04 - 35s/epoch - 8ms/step
Epoch 3/1000
4208/4208 - 34s - loss: 3.9253e-04 - val_loss: 3.4996e-04 - 34s/epoch - 8ms/step
Epoch 4/1000
4208/4208 - 34s - loss: 3.8723e-04 - val_loss: 3.5872e-04 - 34s/epoch - 8ms/step
Epoch 5/1000
4208/4208 - 35s - loss: 3.8433e-04 - val_loss: 3.4191e-04 - 35s/epoch - 8ms/step
Epoch 6/1000
4208/4208 - 34s - loss: 3.8313e-04 - val_loss: 3.4492e-04 - 34s/epoch - 8ms/step
Epoch 7/1000
4208/4208 - 35s - loss: 3.8016e-04 - val_loss: 3.4101e-04 - 35s/epoch - 8ms/step
Epoch 8/1000
4208/4208 - 35s - loss: 3.7840e-04 - val_loss: 3.3715e-04 - 35s/epoch - 8ms/step
Epoch 9/1000
4208/4208 - 35s - loss: 3.7740e-04 - val_loss: 3.3268e-04 - 35s/epoch - 8ms/step
Epoch 10/1000
4208/4208 - 34s - loss: 3.7615e-04 - val_loss: 3.3399e-04 - 34s/epoch - 8ms/step
Epoch 11/1000
4208/4208 - 35s - loss: 3.7630e-04 - val_loss

2024/09/30 06:36:41 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/30 06:36:41 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 41s - loss: 4.0428e-04 - val_loss: 3.4896e-04 - 41s/epoch - 10ms/step
Epoch 2/1000
4208/4208 - 39s - loss: 3.2009e-04 - val_loss: 3.1145e-04 - 39s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 38s - loss: 2.9667e-04 - val_loss: 2.9559e-04 - 38s/epoch - 9ms/step
Epoch 4/1000
4208/4208 - 39s - loss: 2.7895e-04 - val_loss: 2.6649e-04 - 39s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 38s - loss: 2.6807e-04 - val_loss: 2.6396e-04 - 38s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 38s - loss: 2.6090e-04 - val_loss: 2.6034e-04 - 38s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 38s - loss: 2.5659e-04 - val_loss: 2.6284e-04 - 38s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 37s - loss: 2.5303e-04 - val_loss: 2.6084e-04 - 37s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 39s - loss: 2.4988e-04 - val_loss: 2.6536e-04 - 39s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 39s - loss: 2.4752e-04 - val_loss: 2.4825e-04 - 39s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 39s - loss: 2.4517e-04 - val_los

In [ ]:
# regs

for _ in range(100):
    config = {'shape': 22,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}
    
    model = build_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=8, verbose=0)
        mlflow.log_param("CPL", False)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            verbose=2,
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100, verbose=0)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100, verbose=0)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")

